In [1]:
train_file_path=  r"D:\Project\machine_learning_project\housing\artifact\data_ingestion\2022-07-02-12-56-26\ingested_data\train\housing.csv"

In [2]:
import pandas as pd

In [5]:
df = pd.read_csv(train_file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Project\\machine_learning_project\\housing\\artifact\\data_ingestion\\2022-07-02-12-56-26\\ingested_data\\train\\housing.csv'

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
x,y= df.drop(columns=["median_house_value"],axis=1),df[["median_house_value"]]

In [ ]:
x.shape


In [ ]:
x.isna().sum()

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
numerical_column = x.drop(columns=['ocean_proximity'],axis=1)

In [ ]:
numerical_column

In [ ]:
simple_imputer = SimpleImputer(strategy="median")

In [ ]:
simple_imputer.fit_transform(numerical_column)

In [ ]:
df = simple_imputer.transform(numerical_column)

In [ ]:
simple_imputer.feature_names_in_

In [ ]:
simple_imputer.statistics_

In [ ]:
x.latitude.median()

In [ ]:
x.ocean_proximity.value_counts()

In [ ]:
simple_imputer = SimpleImputer(strategy="most_frequent")

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
class Test(BaseEstimator,TransformerMixin):
    
    def __init__(self,strategy="median"):
        self.strategy=strategy

    def fit(self,X):
        self.features_=X.columns
        self.statistic_=[]
        for column in X.columns:
            self.statistic_.append(X[column].median())
        return self


    def transform(self,X:pd.DataFrame):
        for idx,column in enumerate(x.columns):
            X[column].fillna(self.statistic_[idx]) 
        return X




In [ ]:
response =Test().fit(numerical_column)

In [ ]:
response.statistic_

In [ ]:
for idx,column in enumerate(x.columns):
    print(idx,column)

In [ ]:
import numpy as np

In [ ]:
x.columns

In [ ]:
50/25

In [ ]:
x

In [ ]:
x.columns

In [ ]:
COLUMN_TOTAL_ROOMS = "total_rooms"
COLUMN_POPULATION = "population"
COLUMN_HOUSEHOLDS = "households"
COLUMN_TOTAL_BEDROOM = "total_bedrooms"

class FeatureGenerator(BaseEstimator, TransformerMixin):

    def __init__(self, add_bedrooms_per_room=True,
                 total_rooms_ix=3,
                 population_ix=5,
                 households_ix=6,
                 total_bedrooms_ix=4, columns=None):
        """
        FeatureGenerator Initialization
        add_bedrooms_per_room: bool
        total_rooms_ix: int index number of total rooms columns
        population_ix: int index number of total population columns
        households_ix: int index number of  households columns
        total_bedrooms_ix: int index number of bedrooms columns
        """
        try:
            self.columns = columns
            if self.columns is not None:
                total_rooms_ix = self.columns.index(COLUMN_TOTAL_ROOMS)
                population_ix = self.columns.index(COLUMN_POPULATION)
                households_ix = self.columns.index(COLUMN_HOUSEHOLDS)
                total_bedrooms_ix = self.columns.index(COLUMN_TOTAL_BEDROOM)

            self.add_bedrooms_per_room = add_bedrooms_per_room
            self.total_rooms_ix = total_rooms_ix
            self.population_ix = population_ix
            self.households_ix = households_ix
            self.total_bedrooms_ix = total_bedrooms_ix
        except Exception as e:
            raise e

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        try:
            room_per_household = X[:, self.total_rooms_ix] / \
                                 X[:, self.households_ix]
            population_per_household = X[:, self.population_ix] / \
                                       X[:, self.households_ix]
            if self.add_bedrooms_per_room:
                bedrooms_per_room = X[:, self.total_bedrooms_ix] / \
                                    X[:, self.total_rooms_ix]
                generated_feature = np.c_[
                    X, room_per_household, population_per_household, bedrooms_per_room]
            else:
                generated_feature = np.c_[
                    X, room_per_household, population_per_household]

            return generated_feature
        except Exception as e:
            raise e

In [ ]:
arr1=np.arange(10)
arr2=np.arange(10)

In [ ]:
np.c_[arr1,arr2]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
num_pipeline = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy="median")),
    ('feature_genrator',FeatureGenerator()),
    ('scaling',StandardScaler())
])

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
cat_pipeline = Pipeline(steps=[
     ('imputer',SimpleImputer(strategy="most_frequent")),
      ('oneHotEncoder',OneHotEncoder()),
       ('scaling',StandardScaler(with_mean=False))
])

In [ ]:
df=  pd.read_csv(train_file_path)

In [ ]:
df.columns

In [ ]:
num_column_name=['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
        ]

cat_column_name=['ocean_proximity']

In [ ]:
preprocessing  =ColumnTransformer([
    ('num_pipeline',num_pipeline,num_column_name),
    ('cat_pipeline',cat_pipeline,cat_column_name)]
)

In [ ]:
train_df= preprocessing.fit_transform(df)

In [ ]:
test_file_path=r"D:\Project\machine_learning_project\housing\artifact\data_ingestion\2022-07-02-12-56-26\ingested_data\test\housing.csv"

In [ ]:
test_df = pd.read_csv(test_file_path)

In [ ]:
test_df.head()

In [ ]:
test_arr= preprocessing.transform(test_df)

In [ ]:
import dill

In [ ]:
with open('preprocessing.pkl',"wb") as prep_file:
    dill.dump(preprocessing,prep_file)

In [ ]:
with open('preprocessing.pkl', "rb") as file_obj:
    preprocessing_loaded_obj= dill.load(file_obj)

In [ ]:
preprocessing_loaded_obj.transform(test_df)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
x=train_df

In [ ]:
y=np.array(df.median_house_value)

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(x,y)

In [ ]:
lr.fit_intercept

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
y_hat=  lr.predict(x)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(y,y_hat,"o")

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y,y_hat)

In [ ]:
yhat_test = lr.predict(test_arr)

In [ ]:
y_test = np.array(test_df.median_house_value)

In [ ]:
r2_score(y_test,yhat_test)